# Testing the new database I've created in SQLite

## Establishing connection

In [137]:
import sqlite3
from uuid import uuid4
from typing import List, Optional
import pandas as pd
import numpy as np

class SQLConnection:

    def __init__(self, db_name: str = None) -> None:
        self.current_cursor = str(uuid4())
        if db_name is None:
            self.db_name = f'.student_{self.current_cursor}.db'
        else:
            self.db_name = db_name

    def q(self, query: str) -> Optional[List[str]]:
        """Executes a query and returns the result"""
        res = None
        with sqlite3.connect(self.db_name) as con:
            cur = con.cursor()
            for q in query.split(';'):
                try:
                    res = pd.read_sql_query(q.strip(), con)
                except (TypeError, ValueError):
                    pass
        return res

    def connect(self):
        return sqlite3.connect(self.db_name)

database = SQLConnection('ATP_database')

In [96]:
# examining the schema
database.q("SELECT * FROM SQLITE_SCHEMA")

,type,name,tbl_name,rootpage,sql
0,table,MatchPlayer,MatchPlayer,2,CREATE TABLE MatchPlayer (\n\tplayer_id INTEGE...
1,index,sqlite_autoindex_MatchPlayer_1,MatchPlayer,3,None
2,table,Players,Players,4,"CREATE TABLE Players (\n\tplayer_id INTEGER,\n..."
3,table,Level,Level,5,"CREATE TABLE ""Level"" (\n\tlevel_id INTEGER,\n\..."
4,table,Tournaments,Tournaments,6,CREATE TABLE Tournaments (\n\ttourney_id INTEG...
5,table,Matches,Matches,7,"CREATE TABLE Matches (\n\ttourney_id INTEGER,\..."
6,index,sqlite_autoindex_Matches_1,Matches,8,None


## Adding test data into the database

### Creating the test data (players from the 2021 season)

In [144]:
# import information for all players (theres a lot of them)

df_players = pd.read_csv('data/atp_players.csv')
df_players

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
0,100001,Gardnar,Mulloy,R,19131122,USA,185.0,Q54544
1,100002,Pancho,Segura,R,19210620,ECU,168.0,Q54581
2,100003,Frank,Sedgman,R,19271002,AUS,180.0,Q962049
3,100004,Giuseppe,Merlo,R,19271011,ITA,NaN,Q1258752
4,100005,Richard,Gonzalez,R,19280509,USA,188.0,Q53554
...,...,...,...,...,...,...,...,...
56767,211735,David,Saye,U,NaN,USA,NaN,NaN
56768,211736,Patrick,Fletchall,U,NaN,USA,NaN,NaN
56769,211737,Sean,Daryabeigi,U,NaN,USA,NaN,NaN
56770,211738,Jonah,Braswell,U,NaN,USA,NaN,NaN


In [98]:
# import player names from 2021 (not really useful as name is all one column whereas it is 
# split into first and last in df_players)

colnames = ['index', 'player_name']
df_players_2021 = pd.read_csv('data/2021_players.csv', names=colnames)
df_players_2021 = df_players_2021.drop('index', axis='columns')

In [99]:
# importing player_ids for those who played in the 2021 season 

colnames = ['index', 'player_id']
df_2021_player_ids = pd.read_csv('data/2021_player_ids.csv', names=colnames)
df_2021_player_ids = df_2021_player_ids.drop('index', axis='columns')

In [100]:
# merging 2021 player ids with the df_players to return the info about those who 
# played in the 2021 season 

df_2021_player_info = df_2021_player_ids.merge(df_players, on='player_id')

In [101]:
df_2021_player_info.dtypes

player_id        int64
name_first      object
name_last       object
hand            object
dob             object
ioc             object
height         float64
wikidata_id     object
dtype: object

### Using the database connection to insert into the players table 

In [108]:
database.q('SELECT * FROM players')

,player_id,first_name,last_name,hand,height,nationality,dob


In [109]:
# insert all of the players from 2021 into the players table

ids = df_2021_player_info.player_id
firsts = df_2021_player_info.name_first
lasts = df_2021_player_info.name_last
hands = df_2021_player_info.hand
heights = df_2021_player_info.height
nats = df_2021_player_info.ioc
dobs = df_2021_player_info.dob

for id, first, last, hand, height, nat, dob  in zip(ids, firsts, lasts, hands, heights, nats, dobs):
    database.q(f"""
                INSERT INTO players 
                    (player_id, first_name, last_name, hand, height, nationality, dob) 
                VALUES 
                    ('{id}', '{first}', '{last}', '{hand}', '{height}', '{nat}', '{dob}');
                    """)

In [110]:
database.q("SELECT * FROM players")

,player_id,first_name,last_name,hand,height,nationality,dob
0,100644,Alexander,Zverev,R,198,GER,19970420
1,103333,Ivo,Karlovic,R,208,CRO,19790228
2,103499,Aqeel,Khan,R,nan,PAK,19800130
3,103529,Aisam Ul Haq,Qureshi,R,183,PAK,19800317
4,103819,Roger,Federer,R,185,SUI,19810808
...,...,...,...,...,...,...,...
388,209916,Marko,Topo,U,188,SRB,20030913
389,210079,Jisung,Nam,U,nan,KOR,19930815
390,210107,Bor,Artnak,U,nan,SLO,20040604
391,210250,Erik,Arutiunian,U,nan,BLR,20041019


In [199]:
# removing all entries in the players table 

database.q("DELETE FROM players")
database.q("SELECT * FROM players")

,player_id,first_name,last_name,hand,height,nationality,dob


### 

### Dealing with Null data in the players dataframe

In [125]:
df_players.isna().sum()

player_id          0
name_first       175
name_last         42
hand             243
dob            11828
ioc               54
height         53989
wikidata_id    51880
dtype: int64

In [126]:
df_players.shape

(56772, 8)

In [129]:
df_players[df_players.name_first.isna()]

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
10429,110444,NaN,NaN,U,NaN,UNK,NaN,NaN
17067,117083,NaN,Rf Le Sueur,U,NaN,RSA,NaN,NaN
17093,117109,NaN,N Zaher,U,NaN,EGY,NaN,NaN
17155,117171,NaN,Ev Bobb,U,NaN,IND,NaN,NaN
25629,125645,NaN,I Bunea,U,NaN,ROU,NaN,NaN
...,...,...,...,...,...,...,...,...
49714,204680,NaN,Mezquita,R,NaN,ESP,NaN,NaN
49715,204681,NaN,Vicens,R,NaN,UNK,NaN,NaN
49722,204688,NaN,Segun,R,NaN,UNK,NaN,NaN
51999,206965,NaN,I Georgiadis,NaN,NaN,GRE,NaN,NaN


In [145]:
df_players[df_players.name_first == 'Wanaro']

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
192,100193,Wanaro,N'Godrella,R,19491019,FRA,NaN,Q3566000
47615,202581,Wanaro,Ngodrella,U,19491018,FRA,NaN,Q3566000


In [ ]:
# remove the wikidata_id columns as it is full of missing data and I won't be using it 

df_players = df_players.drop('wikidata_id', axis=1)

### COMMENTS
* Populated the database fine and deleted the data again also fine (only populated the players table)
    * Had to put the variables in apostrophes e.g. `'{id}', '{first}',` etc.
* Yet to automate it (fully) but managed to loop through a dataframe and populate the table that way 
    * There could be a way to add all the data in one SQL query which could be 'better' but not sure 
* Started to remove null/useless data from the players table

### NEXT STEPS
* Work out how to add the data into the tables where if the entry is NaN it adds the entry anyway just with a null value in that column ✅
* Establish what U means for handedness - means ambidextrous ✅
* Set up a pipeline that will take the players csv, convert into a dataframe, make the data usable (deal with NaN values) and add to the players table 
    * Then tweak the pipeline to take a new csv with broadly the same data but maybe a few extra entries, and add the updated data to the data frame with no duplicated obviously 
    * Thinking with the players data could just delete the whole table and replace with the new updated csv I've been sent (carrying out the same transforming process every time)
    * This won't work for the other tables as I will be adding separate csv files together which should mean there will be no issue with duplication as tourney_id will be different every time so should be able to just `INSERT INTO table` no problem
* Create the pipeline for the other tables based on that of the players table (will be slightly more involved I think, especially the transform process)

In [195]:
# replacing all nan values with 'Null' to make it easier to deal with and saving as a new dataframe in case it doesn't work
df_players_null = df_players.replace(np.nan, 'Null')

In [196]:
# this deals specifically with Jason who has a nickname JJ that is in "" in the name_first column

df_players_null.name_first = df_players_null.name_first.str.replace('"', "'")
df_players_null[df_players_null.player_id == 140087]

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
40071,140087,Jason 'Jj',Belan,R,19750604,GER,Null,Null


In [198]:
# replacing NaN with Null allows the data to be inserted into the players table with Null data where it is missing

ids = df_players_null.player_id
firsts = df_players_null.name_first
lasts = df_players_null.name_last
hands = df_players_null.hand
heights = df_players_null.height
nats = df_players_null.ioc
dobs = df_players_null.dob

for id, first, last, hand, height, nat, dob  in zip(ids, firsts, lasts, hands, heights, nats, dobs):
    database.q(f"""
                INSERT INTO players 
                    (player_id, first_name, last_name, hand, height, nationality, dob) 
                VALUES 
                    ("{id}", "{first}", "{last}", "{hand}", "{height}", "{nat}", "{dob}");
                    """)

In [185]:
id_list = list(df_players_null.wikidata_id)
duplicate_ids = []
for id in id_list:
    if id != 'Null' and id_list.count(id) > 1:
        duplicate_ids.append(id)

In [200]:
df_players_null[df_players_null.wikidata_id.isin(duplicate_ids)].sort_values(by='wikidata_id')

,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id
2320,102321,Lazaro,Navarro,R,19740128,CUB,Null,Q102191659
2319,102320,Lazaro,Navarro Batles,R,19740128,CUB,Null,Q102191659
3904,103905,Ricardo,Chile Fonte,U,19820107,CUB,Null,Q102346559
3905,103906,Ricardo,Chile,U,19820107,CUB,Null,Q102346559
8636,108651,Sandor,Martinez Breijo,R,19810321,CUB,Null,Q102347405
...,...,...,...,...,...,...,...,...
13831,113847,Maurice,Ferrier,U,19011108,SUI,Null,Q97152359
3030,103031,Julian,Alonso,R,19770802,ESP,185.0,Q980373
31347,131363,Julian,Alonso,R,19770802,ESP,Null,Q980373
48809,203775,Roland,So,U,19660622,PHI,Null,Q99526930


* Need to work out how to choose the entry with more information for the wikidata_id duplicates, and then remove the others (without eyeballing e.g. with potential to automate)

* See if there are any other cases of missing data or null data that can be addressed?

* Then set up the pipeline to take the players data and add them all to the database table 